In [105]:
from netgen.occ import *
import numpy as np
from ngsolve import *
from ngsolve.webgui import Draw
maxh = 1/10



freq = 50
mu0 = 4e-7*np.pi
mu_Fe = mu0 *1000
mu_Air = mu0

sigma_Fe =2e6


omega = freq*2*np.pi

delta = np.sqrt(2/(sigma_Fe*omega*mu_Fe))


In [106]:
Nsheets = 2
ff = 0.9
d = delta/2



order0 = 2

B0 = 1

In [107]:
dFe = d*ff
d0 = d*(1-ff)

H_core = Nsheets*dFe + (Nsheets-1)*d0
W_core = H_core

W = 2*W_core
H = 2*H_core

wp = WorkPlane()
outer = wp.RectangleC(W, H).Face()
outer.name = "air"
outer.edges.Max(X).name = "right"
outer.edges.Min(X).name = "left"
outer.edges.Max(Y).name = "top"
outer.edges.Min(Y).name = "bottom"


rec_sheets =[]
x_pos = - W_core/2


for i in range(Nsheets):
    wp.MoveTo(x_pos, -H_core/2)

    rec_sheets.append(wp.Rectangle(dFe, H_core).Face())
    rec_sheets[-1].name = f"iron{i}"


    x_pos += d

print(x_pos, W_core/2)

rec_sheets = Glue(rec_sheets)
rec_sheets.edges.maxh = delta/10

geo = Glue([outer - rec_sheets, rec_sheets])



meshRef = Mesh(OCCGeometry(geo, dim=2).GenerateMesh(maxh=delta))
#Draw(meshRef.MaterialCF({"iron":1, "air":2}, default=0), meshRef)
Draw(x, meshRef)


print("domains", set(meshRef.GetMaterials()))
print("bnds", set(meshRef.GetBoundaries()))

print("penetration depth", delta)

0.0008355634512324505 0.0007559859796865029


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

domains {'iron0', 'air', 'iron1'}
bnds {'default', 'bottom', 'top', 'left', 'right'}
penetration depth 0.0015915494309189533


In [108]:
mu = meshRef.MaterialCF({"iron.*":mu_Fe, "air":mu_Air}, default=0)
sigma = meshRef.MaterialCF({"iron.*":sigma_Fe}, default=0)


nu = 1/mu
rho = 1/sigma

In [134]:
excitation_orientation = "y"

# ------------------------------------------------------------------------------
# --- Excitation
# ------------------------------------------------------------------------------
H0_amp = 1

# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# +++ reference solution
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
print("order0", order0)
print("numSheets", Nsheets)

if excitation_orientation == "x":
    dir_A = "top|bottom"
else:
    dir_A = "left|right"
VA = H1(meshRef,order=order0, complex=True, dirichlet=dir_A)
VNum = []
for i in range(Nsheets):
    VNum.append(NumberSpace(meshRef, complex=True, definedon=meshRef.Materials(f"iron{i}")))

V = FESpace([VA] + VNum)
ndof = V.ndof	
print(f"VA  :{VA.ndof}")    
print(f"ndof  :{ndof}")    

# Gridfunctions
sol_ref = GridFunction(V, "sol") 
A_ref = sol_ref.components[0] 

trials = V.TrialFunction()
tests  = V.TestFunction()

uA = trials[0]
vA = tests[0]


# ------------------------------------------------------------------------------
# Matrix
# ------------------------------------------------------------------------------
with TaskManager():
    # Bilinear form with 
    ah_ref = BilinearForm(V, symmetric=True)

    # A:
    ah_ref += nu*grad(uA) * grad(vA) * dx
    ah_ref += 1j*omega * sigma * uA * vA * dx("iron.*") #ok


    # for i in range(Nsheets):
    #     ah_ref += 1j *  sigma * omega *  uA * vA * trials[i+1] * dx(f"iron{i}")
    ah_ref +=  1e6 * uA * vA * trials[1] * tests[1] * dx(f"iron0")
    ah_ref +=  1e6 * uA * vA * trials[2] * tests[2] * dx(f"iron1")

    prec = Preconditioner(ah_ref, type = "direct")  
    ah_ref.Assemble()

    # excitation on the right side
    f_ref = LinearForm(V) 
    #    f_ref += H0_amp * ex * curl(v_ref) * dx("air|iron")

    f_ref.Assemble()

    print(ah_ref.mat.AsVector().Norm())
# ------------------------------------------------------------------------------
# ------ Solve It
# ------------------------------------------------------------------------------
with TaskManager():
    
    if excitation_orientation == "x":
        A_ref.Set(B0*y, BND)
    else:
        A_ref.Set(B0*x, BND)
    # solve it
    solvers.BVP(bf=ah_ref, lf=f_ref, gf=sol_ref, pre=prec, maxsteps=5)
            
    gradA = grad(A_ref)
    B = CF((gradA[1], -gradA[0])) 
    H = 1/mu * B_ref
    E = -1j*omega*(A_ref )
    J = sigma * E_ref

    hb = 1/2 * InnerProduct(H,  B) / 2
    p =  1/2 * InnerProduct(E,  J) 

#Draw(B.real, meshRef, vectors=True)
#Draw(J.imag, meshRef)
Draw(sol_ref.components[1], meshRef)


sol_ref.components[1]

order0 2
numSheets 2
VA  :720
ndof  :722
30952050.846712947
CG iteration 1, residual = 4.830434646696971     
CG iteration 2, residual = 4.020734897712075e-15     


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…